# TMDB PySpark Analysis
This notebook replicates the TMDB analysis originally done in Pandas, using PySpark instead.

In [17]:
# tmdb_functions.py

import requests
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, when
from pyspark.sql.types import StringType, FloatType, IntegerType, StructType, StructField, ArrayType
from dotenv import load_dotenv
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_202"

# -----------------------------------------
# 1. Start a Spark Session
# -----------------------------------------
def create_spark_session(app_name="TMDB Movie Analysis"):
    spark = SparkSession.builder \
        .appName(app_name) \
        .getOrCreate()
    return spark

# -----------------------------------------
# 2. Fetch data from TMDB API
# -----------------------------------------

load_dotenv()  # Looks for .env in same directory
api_key = os.getenv("API_KEY")

movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397,
    420818, 24428, 168259, 99861, 284054, 12445,
    181808, 330457, 351286, 109445, 321612, 260513
]
def fetch_movie_data(movie_ids, api_key):
    movie_data = []

    for movie_id in movie_ids:
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US&append_to_response=credits"
        response = requests.get(url)
        if response.status_code == 200:
            movie_data.append(response.json())
        else:
            print(f"Failed to fetch movie ID {movie_id}: {response.status_code}")

    return pd.DataFrame(movie_data)

df_movies = fetch_movie_data(movie_ids, api_key)

# Preview first few rows
print(df_movies.head())





Failed to fetch movie ID 0: 404
   adult                     backdrop_path   
0  False  /7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg  \
1  False  /vL5LR6WdxWPjLPFRLe133jXWsh5.jpg   
2  False  /k6EOrckWFuz7I4z4wiRwz8zsj4H.jpg   
3  False  /mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg   
4  False  /sCzcYW9h55WcesOqA12cgEr9Exw.jpg   

                               belongs_to_collection     budget   
0  {'id': 86311, 'name': 'The Avengers Collection...  356000000  \
1  {'id': 87096, 'name': 'Avatar Collection', 'po...  237000000   
2  {'id': 10, 'name': 'Star Wars Collection', 'po...  245000000   
3  {'id': 86311, 'name': 'The Avengers Collection...  300000000   
4                                               None  200000000   

                                              genres   
0  [{'id': 12, 'name': 'Adventure'}, {'id': 878, ...  \
1  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   
2  [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...   
3  [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...   
4  [{'id

In [7]:
spark = create_spark_session()
spark_df = spark.createDataFrame(df_movies)


In [8]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType

cast_schema = ArrayType(
    StructType([
        StructField("name", StringType()),
        StructField("order", IntegerType())
    ])
)

crew_schema = ArrayType(
    StructType([
        StructField("name", StringType()),
        StructField("job", StringType())
    ])
)

credits_schema = StructType([
    StructField("cast", cast_schema),
    StructField("crew", crew_schema)
])


In [9]:
from pyspark.sql.functions import size, expr

# Rename nested credit columns directly (NO from_json needed)
spark_df = spark_df.withColumn("cast", col("credits.cast"))
spark_df = spark_df.withColumn("crew", col("credits.crew"))

# Cast size
spark_df = spark_df.withColumn("cast_size", size(col("cast")))
spark_df = spark_df.withColumn("crew_size", size(col("crew")))

# Extract Director's name from crew
spark_df = spark_df.withColumn(
    "director",
    expr("filter(crew, x -> x.job = 'Director')[0].name")
)


In [10]:
spark_df.select("credits").printSchema()


root
 |-- credits: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: boolean (valueContainsNull = true)



In [12]:
from pyspark.sql.functions import col, when, udf, expr
from pyspark.sql.types import StringType

# UDF to extract pipe-separated names from array of structs
def extract_names(x):
    try:
        return "|".join([i['name'] for i in x]) if x else None
    except:
        return None

extract_names_udf = udf(extract_names, StringType())

def clean_movie_df(df):
    # 1. Parse nested JSON-like fields into pipe-separated strings
    cols_to_parse = ["genres", "production_companies", "production_countries", "spoken_languages"]
    for field in cols_to_parse:
        df = df.withColumn(field, extract_names_udf(col(field)))

    # Parse belongs_to_collection to get collection name
    df = df.withColumn("belongs_to_collection", col("belongs_to_collection.name"))

    # 2. Clean numeric and text fields
    df = df.withColumn("budget", col("budget").cast("float") / 1_000_000)
    df = df.withColumn("revenue", col("revenue").cast("float") / 1_000_000)
    df = df.withColumn("runtime", col("runtime").cast("float"))

    # Replace 0s with null
    for col_name in ["budget", "revenue", "runtime"]:
        df = df.withColumn(col_name, when(col(col_name) == 0, None).otherwise(col(col_name)))

    # Replace placeholder text with null
    for col_name in ["tagline", "overview"]:
        df = df.withColumn(col_name, when(col(col_name).isin("No Data", "", "N/A"), None).otherwise(col(col_name)))

    # Optional cleanup: drop 'video', 'adult', etc., if needed (can uncomment later)
    # df = df.drop("video", "adult", "homepage", "original_title", "imdb_id")

    return df


In [15]:
from pyspark.sql.functions import col, when, expr, array_join, transform

def clean_movie_df(df):
    # 1. Convert JSON-like array fields to pipe-separated strings
    cols_to_parse = ["genres", "production_companies", "production_countries", "spoken_languages"]
    for field in cols_to_parse:
        df = df.withColumn(
            field,
            array_join(transform(col(field), lambda x: x["name"]), "|")
        )

    # Extract collection name from belongs_to_collection
    df = df.withColumn("belongs_to_collection", col("belongs_to_collection.name"))

    # 2. Handle numeric fields
    df = df.withColumn("budget", col("budget").cast("float") / 1_000_000)
    df = df.withColumn("revenue", col("revenue").cast("float") / 1_000_000)
    df = df.withColumn("runtime", col("runtime").cast("float"))

    for col_name in ["budget", "revenue", "runtime"]:
        df = df.withColumn(col_name, when(col(col_name) == 0, None).otherwise(col(col_name)))

    # 3. Replace placeholders in text
    for col_name in ["tagline", "overview"]:
        df = df.withColumn(
            col_name,
            when(col(col_name).isin("No Data", "", "N/A"), None).otherwise(col(col_name))
        )

    return df


In [13]:
cleaned_df = clean_movie_df(spark_df)
cleaned_df.printSchema()
cleaned_df.select("genres", "budget", "tagline").show(truncate=False)


root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: long (nullable = true)
 |-- budget: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: long (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- origin_country: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- runtime: float (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title:

Py4JJavaError: An error occurred while calling o157.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [11]:
from pyspark.sql.functions import col, when, expr, array_join, transform

def clean_movie_df(df):
    # 1. Convert nested fields to pipe-separated strings (NO UDFs)
    nested_fields = ["genres", "production_companies", "production_countries", "spoken_languages"]
    for field in nested_fields:
        df = df.withColumn(
            field,
            array_join(
                transform(col(field), lambda x: x["name"]),
                "|"
            )
        )

    # 2. Extract collection name
    df = df.withColumn("belongs_to_collection", col("belongs_to_collection.name"))

    # 3. Clean budget/revenue/runtime (convert to million USD and handle 0s)
    for field in ["budget", "revenue"]:
        df = df.withColumn(field, col(field).cast("float") / 1_000_000)
        df = df.withColumn(field, when(col(field) == 0, None).otherwise(col(field)))
    df = df.withColumn("runtime", when(col("runtime") == 0, None).otherwise(col("runtime").cast("float")))

    # 4. Clean text fields with bad placeholders
    for field in ["tagline", "overview"]:
        df = df.withColumn(field, when(col(field).isin("No Data", "", "N/A"), None).otherwise(col(field)))

    return df


In [12]:
spark = create_spark_session()
spark_df = spark.createDataFrame(df_movies)
cleaned_df = clean_movie_df(spark_df)
cleaned_df.select("genres", "budget", "tagline").show(truncate=False)


Py4JJavaError: An error occurred while calling o303.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [45]:
type(df_movies.iloc[0]["credits"])


dict

In [40]:
from pyspark.sql.functions import col, to_date

# Convert pandas DataFrame to list of dicts
movie_data = df_movies.to_dict(orient="records")

# Create Spark DataFrame from structured schema
df_spark = spark.createDataFrame(movie_data, schema=tmdb_schema)

# Cast release_date to Spark DateType
df_spark = df_spark.withColumn("release_date", to_date("release_date"))

# Try a simple select
df_spark.select("id", "title", "release_date").show(5)


Py4JJavaError: An error occurred while calling o651.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 13) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheck

In [15]:
df_spark = spark.createDataFrame(df_movies, schema=tmdb_schema)
df_spark = df_spark.withColumn("release_date", col("release_date").cast("date"))

df_spark.printSchema()
df_spark.show(5)


root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: struct (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- poster_path: string (nullable = true)
 |    |-- backdrop_path: string (nullable = true)
 |-- budget: long (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- origin_country: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: array (nullable

Py4JJavaError: An error occurred while calling o453.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 7) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [20]:
movie_data = df_movies.to_dict(orient="records")

import json
print(json.dumps(movie_data[0], indent=2))



{
  "adult": false,
  "backdrop_path": "/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg",
  "belongs_to_collection": null,
  "budget": 356000000,
  "genres": null,
  "homepage": "https://www.marvel.com/movies/avengers-endgame",
  "id": 299534,
  "imdb_id": "tt4154796",
  "origin_country": [
    "US"
  ],
  "original_language": "en",
  "original_title": "Avengers: Endgame",
  "overview": "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.",
  "popularity": 53.8808,
  "poster_path": "/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg",
  "production_companies": null,
  "production_countries": null,
  "release_date": "2019-04-24",
  "revenue": 2799439100,
  "runtime": 181,
  "spoken_languages": null,
  "status": "Released",
  "tagline": "Avenge the 

In [13]:
import json
print(json.dumps(df_movies.iloc[0].to_dict(), indent=2))


{
  "adult": false,
  "backdrop_path": "/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg",
  "belongs_to_collection": null,
  "budget": 356000000,
  "genres": null,
  "homepage": "https://www.marvel.com/movies/avengers-endgame",
  "id": 299534,
  "imdb_id": "tt4154796",
  "origin_country": [
    "US"
  ],
  "original_language": "en",
  "original_title": "Avengers: Endgame",
  "overview": "After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.",
  "popularity": 53.8808,
  "poster_path": "/ulzhLuWrPK07P1YkdWQLZnQh1JL.jpg",
  "production_companies": null,
  "production_countries": null,
  "release_date": "2019-04-24",
  "revenue": 2799439100,
  "runtime": 181,
  "spoken_languages": null,
  "status": "Released",
  "tagline": "Avenge the 

In [12]:
df_movies.iloc[0]


adult                                                                False
backdrop_path                             /7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
belongs_to_collection                                                 None
budget                                                           356000000
genres                                                                None
homepage                    https://www.marvel.com/movies/avengers-endgame
id                                                                  299534
imdb_id                                                          tt4154796
origin_country                                                        [US]
original_language                                                       en
original_title                                           Avengers: Endgame
overview                 After the devastating events of Avengers: Infi...
popularity                                                         53.8808
poster_path              

In [9]:
def flatten_json_columns(df):
    import pandas as pd

    def extract_list_names(json_list):
        """Convert list of dicts to pipe-separated string."""
        if isinstance(json_list, list):
            return "|".join([entry.get("name", "") for entry in json_list])
        return None

    def extract_dict_name(entry):
        """Convert single dict to name (e.g. for belongs_to_collection)."""
        if isinstance(entry, dict):
            return entry.get("name")
        return None

    # Apply to each column
    df['genres'] = df['genres'].apply(extract_list_names)
    df['spoken_languages'] = df['spoken_languages'].apply(extract_list_names)
    df['production_companies'] = df['production_companies'].apply(extract_list_names)
    df['production_countries'] = df['production_countries'].apply(extract_list_names)
    df['belongs_to_collection'] = df['belongs_to_collection'].apply(extract_dict_name)

    return df


df_movies_cleaned = flatten_json_columns(df_movies)
df_movies_cleaned.head(2)


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,origin_country,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,credits
0,False,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg,None,356000000,None,https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,[US],en,...,2799439100,181,None,Released,Avenge the fallen.,Avengers: Endgame,False,8.237,26247,"{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,None,237000000,None,https://www.avatar.com/movies/avatar,19995,tt0499549,[US],en,...,2923706026,162,None,Released,Enter the world of Pandora.,Avatar,False,7.588,32162,"{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [11]:
df_movies_cleaned.dtypes

adult                       bool
backdrop_path             object
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                         int64
imdb_id                   object
origin_country            object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                    int64
runtime                    int64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                       bool
vote_average             float64
vote_count                 int64
credits                   object
dtype: object

In [10]:
df_spark = spark.createDataFrame(df_movies_cleaned)
df_spark.show(5)  # This should now work 100%


PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.

In [4]:
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_202"  # Replace with your actual JDK path

from pyspark.sql import SparkSession

def create_spark_session(app_name="TMDB Movie Analysis"):
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .getOrCreate()

spark = create_spark_session()


In [5]:
spark.range(10).show()


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [6]:
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_202"  # Ensure Java path is known

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, year
from pyspark.sql.types import StringType, FloatType, IntegerType

# Start Spark
def create_spark_session(app_name="TMDB Movie Analysis"):
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .getOrCreate()

spark = create_spark_session()

# -------- UDF for flattening nested fields --------
def extract_names(json_list):
    if json_list and isinstance(json_list, list):
        return "|".join([entry.get("name") for entry in json_list if isinstance(entry, dict) and "name" in entry])
    return None

extract_names_udf = udf(extract_names, StringType())

# -------- Data Cleaning Function --------
def clean_movie_data(spark, df_pd):
    """
    Takes a Pandas DataFrame of raw movie data and returns a cleaned Spark DataFrame.
    """

    # Step 1: Convert to Spark DataFrame
    df = spark.createDataFrame(df_pd)

    # Step 2: Drop irrelevant columns
    columns_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
    df = df.drop(*columns_to_drop)

    # Step 3: Flatten JSON-like columns (skip credits for now)
    df = df \
        .withColumn("genres", extract_names_udf(col("genres"))) \
        .withColumn("spoken_languages", extract_names_udf(col("spoken_languages"))) \
        .withColumn("production_countries", extract_names_udf(col("production_countries"))) \
        .withColumn("production_companies", extract_names_udf(col("production_companies"))) \
        .withColumn("belongs_to_collection", extract_names_udf(col("belongs_to_collection")))

    # Step 4: Convert column types
    df = df.withColumn("budget", col("budget").cast(FloatType())) \
           .withColumn("revenue", col("revenue").cast(FloatType())) \
           .withColumn("popularity", col("popularity").cast(FloatType())) \
           .withColumn("vote_count", col("vote_count").cast(IntegerType())) \
           .withColumn("vote_average", col("vote_average").cast(FloatType())) \
           .withColumn("runtime", col("runtime").cast(FloatType()))

    # Step 5: Replace invalid values with null
    df = df.withColumn("budget", when(col("budget") == 0, None).otherwise(col("budget"))) \
           .withColumn("revenue", when(col("revenue") == 0, None).otherwise(col("revenue"))) \
           .withColumn("runtime", when(col("runtime") == 0, None).otherwise(col("runtime")))

    # Step 6: Format release_date and extract year
    df = df.withColumn("release_date", col("release_date").cast("date")) \
           .withColumn("release_year", year(col("release_date")))

    # Step 7: Convert to million USD
    df = df.withColumn("budget_musd", col("budget") / 1_000_000) \
           .withColumn("revenue_musd", col("revenue") / 1_000_000)

    # Step 8: Remove invalid records
    df = df.filter(col("id").isNotNull() & col("title").isNotNull())

    # Step 9: Filter for 'Released' movies
    df = df.filter(col("status") == "Released").drop("status")

    # Step 10: Select and reorder columns
    ordered_columns = [
        "id", "title", "tagline", "release_date", "release_year", "genres", "belongs_to_collection",
        "original_language", "budget_musd", "revenue_musd", "production_companies",
        "production_countries", "vote_count", "vote_average", "popularity", "runtime",
        "overview", "spoken_languages", "poster_path"
    ]

    df = df.select(*[c for c in ordered_columns if c in df.columns])

    return df


df_clean = clean_movie_data(spark, df_movies)
df_clean.show(5)


Py4JJavaError: An error occurred while calling o393.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 5) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [7]:
spark = create_spark_session()

# Just convert your original Pandas DataFrame and try showing it
df_raw_spark = spark.createDataFrame(df_movies)

# Try to print schema and show a few rows
df_raw_spark.printSchema()
df_raw_spark.show(5)



root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- budget: long (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- homepage: string (nullable = true)
 |-- id: long (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- origin_country: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull

Py4JJavaError: An error occurred while calling o420.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 6) (DESKTOP-D8B3G78 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi